In [1]:
import numpy as np
import torch
import os
import gc
from sklearn import preprocessing

In [2]:
path = '/home/jua/corsika/run/energy_var/npz/square/all/'
filelist = os.listdir(path)
gbins = {}
pbins = {}
for file in filelist:
    n = int(file.split('_')[2][-1])
    if file.split("_")[0]=='gamma':
        gbins[n]=np.load(path+file)
    if file.split("_")[0]=='proton':
        pbins[n]=np.load(path+file)

In [3]:
def norm(Ebin, g, p, normtype):
    g_label = torch.zeros([len(g)])
    p_label = torch.ones([len(p)])
    if normtype == "minmax":
        min_max_scaler = preprocessing.MinMaxScaler()
        for j in range(len(g)):
            for i in range(4):
                g[j][i] = min_max_scaler.fit_transform(g[j][i])
        for j in range(len(p)):
            for i in range(4):
                p[j][i] = min_max_scaler.fit_transform(p[j][i])
    elif normtype == "std":
        for j in range(len(g)):
            for i in range(4):
                scaler = preprocessing.StandardScaler().fit(g[j][i])
                g[j][i] = scaler.transform(g[j][i])
        for j in range(len(p)):
            for i in range(4):
                scaler = preprocessing.StandardScaler().fit(p[j][i])
                p[j][i] = scaler.transform(p[j][i])
    images = np.vstack((g,p))
    labels = np.hstack((g_label, p_label))
    np.savez_compressed("/home/jua/SWGO/square/data/{}/bin/bin{}.npz".format(normtype, Ebin),image=images,label=labels)
    return g, p

In [6]:
for i in range(2,9):
    norm(i,gbins[i]['image'][:],pbins[i]['image'][:],'std')

In [ ]:
for i in range(1,9):
    norm(i,gbins[i]['image'][:],pbins[i]['image'][:],'minmax')